In [ ]:
import pandas as pd
import os
from io import StringIO
import plotly.express as px

all_dataframes = []

for filename in os.listdir('.'):
    if filename.startswith('BFIAMU_') and filename.endswith('.csv'):
        try:
            date_str = filename.split('_')[1].split('.')[0]
            date_dt = pd.to_datetime(date_str, format='%Y%m%d')

            with open(filename, 'r', encoding='windows-1252') as file:
                lines = file.readlines()[1:]
            remark_index = next((i for i, line in enumerate(lines) if line.startswith('Remark:')), len(lines))
            valid_lines = lines[:remark_index]

            csv_data = StringIO(''.join(valid_lines))
            df = pd.read_csv(csv_data)
            df.dropna(axis=1, how='all', inplace=True)
            df.insert(0, 'Date', date_dt)

            all_dataframes.append(df)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

combined_df = pd.concat(all_dataframes, ignore_index=True)
combined_df.sort_values(by='Date', inplace=True)

food_df = combined_df[combined_df['Category of Index'] == 'Food'].copy()
food_df['Trade Volume'] = food_df['Trade Volume'].str.replace(',', '').astype(float)

In [ ]:
fig = px.line(food_df, x='Date', y='Trade Volume', title='Trade Volume Over Time for Food Category (Sorted by Date)')
fig.show()